In [ ]:

from google.colab import drive
drive.mount('/drive')

# Installing Libraries
!pip install -U pip setuptools wheel
!pip install -U spacy[cuda101,transformers,lookups]
!python -m spacy download en_core_web_lg

Mounted at /drive
     |████████████████████████████████| 1.5MB 10.9MB/s 
Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (53.0.0)
Requirement already up-to-date: wheel in /usr/local/lib/python3.6/dist-packages (0.36.2)
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 12.8 MB 45 kB/s 
     |████████████████████████████████| 9.2 MB 53.1 MB/s 
     |████████████████████████████████| 1.1 MB 43.5 MB/s 
     |████████████████████████████████| 456 kB 45.0 MB/s 
     |████████████████████████████████| 93.4 MB 17 kB/s 
     |████████████████████████████████| 1.8 MB 46.9 MB/s 
     |████████████████████████████████| 978 kB 38.1 MB/s 
     |████████████████████████████████| 66 kB 4.2 MB/s 
     |████████████████████████████████| 100 kB 9.5 MB/s 
     |████████████████████████████████| 2.9 MB 42.6 MB/s 
     |████████████████████████████████| 883 kB 40.4 MB/

In [ ]:
import spacy
import pandas as pd

import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path

In [ ]:
output_dir = '/drive/MyDrive/LDW 2021/Output/Model/'
if output_dir is not None:
    output_dir = Path(output_dir)

df = pd.read_csv('/drive/MyDrive/LDW 2021/Output/CleanData.csv')

df.head()

,RecordID,text,pubDate,publisher,type
0,93258144,lair atlnu no wesl ststst betweenand sthav mar...,1873-02-19,New York Times Company,Classified Advertisement
1,95235433,beecham's pills for constipation ioo and get t...,1895-03-03,New York Times Company,Advertisement
2,95157230,phe peruan hub line i out all staring signs hi...,1893-09-03,New York Times Company,Advertisement
3,95437997,why not a music box for your holiday present a...,1896-12-13,New York Times Company,Advertisement
4,91760250,g new wo rh by sorace greely the attieitican c...,1863-11-05,New York Times Company,Classified Advertisement


In [ ]:
# Beginning Custom NLP With Spacy

spacy.prefer_gpu()

print("Loading from:", output_dir)
nlp = spacy.load(output_dir)

print(nlp.pipe_names)

Loading from: /drive/MyDrive/LDW 2021/Output/Model
['tok2vec', 'tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer']


In [ ]:
# Note: to apply nlp to whole column

# df['doc'] = [nlp(text) for text in df.text]

df.sample(10)

SyntaxError: ignored

In [ ]:
# Noun Chunks
for chunk in df.doc[1].noun_chunks:
  print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text)

beecham's pills pills ROOT pills
constipation ioo ioo pobj for
the book book dobj get
your druggist druggist pobj at
it it pobj by
eocco reed reed pobj than
bart bart conj reed
silversmiths silversmiths pobj on
e iiio iiio appos silversmiths
sileir sileir pobj in
ll ll pobj w
reasonable prices t union suare new yori yori pobj at


In [ ]:
# Entity Checking
for ent in df.doc[1].ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

beecham's 0 9 ORG
pills 10 15 PRODUCT


In [ ]:
spacy.displacy.render(df.doc[10], style='ent', jupyter=True)

In [ ]:
df.head()

,RecordID,text,pubDate,publisher,type,doc
0,93258144,lair atlnu no wesl ststst betweenand sthav mar...,1873-02-19,New York Times Company,Classified Advertisement,"(lair, atlnu, no, wesl, ststst, betweenand, st..."
1,95235433,beecham's pills for constipation ioo and get t...,1895-03-03,New York Times Company,Advertisement,"(beecham, 's, pills, for, constipation, ioo, a..."
2,95157230,phe peruan hub line i out all staring signs hi...,1893-09-03,New York Times Company,Advertisement,"(phe, peruan, hub, line, i, out, all, staring,..."
3,95437997,why not a music box for your holiday present a...,1896-12-13,New York Times Company,Advertisement,"(why, not, a, music, box, for, your, holiday, ..."
4,91760250,g new wo rh by sorace greely the attieitican c...,1863-11-05,New York Times Company,Classified Advertisement,"(g, new, wo, rh, by, sorace, greely, the, atti..."


In [ ]:
# for i in range(len(df)):
#   for entity in df.doc[i].ents:
#           df['entities'] = [entity.text, entity.label_]
for row in df.doc:
  for ent in row.doc.ents:
    df['entities'] = df.append([ent.text, ent.label_], ignore_index=True)

ValueError: ignored

In [ ]:
# Parts of Speech Parsing
for token in df.spell[1]:
  print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

In [ ]:
for token in df.spell[1]:
    print(token.text, token.has_vector, token.vector_norm, token.is_oov)

In [ ]:
# Use this for any confusing acronyms
spacy.explain('GPE')

In [ ]:
# Testing Function to Extract Nouns

def extract_nouns(doc):
    pos = [tok.i for tok in doc if tok.pos_ == "NOUN" or "PROPN"]
    consecutives = []
    current = []
    for elt in pos:
        if len(current) == 0:
            current.append(elt)
        else:
            if current[-1] == elt - 1:
                current.append(elt)
            else:
                consecutives.append(current)
                current = [elt]
    if len(current) != 0:
        consecutives.append(current)
    return [doc[consecutive[0]:consecutive[-1]+1] for consecutive in consecutives]

In [ ]:
with open('/drive/MyDrive/LDW 2021/Output/AdData_Cleaned.csv', 'a') as output:
    writer = csv.DictWriter(output, fieldnames = df.columns)
    writer.writeheader()
    writer = csv.writer(output)

    curr_ind = 0

    # splitting data to 330 * 5000 batches to prevent overloading RAM
    for iter in range(330): # change to smaller batch size if you want to test
        batch_count = 5000
        cleaned_data = []

        for data in df.loc[curr_ind:curr_ind + batch_count, :].values.tolist():    
            output = [data[0]]
            clean_text = re.sub(r'\s\s+', ' ', data[1]) # removing spaces
            output.append(clean_text)
            output.extend(data[2:])

            cleaned_data.append(output)

        writer.writerows(cleaned_data)
        del cleaned_data
        curr_ind += batch_count

        print(iter) # monitor progress